### Exploring precision and recall

The goal of this assignment is to understand precision-recall in the context of classifiers.

Use Amazon review data in its entirety.

Train a logistic regression model.

Explore various evaluation metrics: accuracy, confusion matrix, precision, recall.

Explore how various metrics can be combined to produce a cost of making an error.

Explore precision and recall curves.

NOTE: Someth

In [142]:
import numpy as np
import pandas as pd
import string
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style()

In [143]:
products = pd.read_csv('amazon_baby.csv')

In [144]:
products.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


### Perform text cleaning

We start by removing punctuation, so that words "cake." and "cake!" are counted as the same word.

1. Write a function remove_punctuation that strips punctuation from a line of text
2. Apply this function to every element in the review column of products, and save the result to a new column review_clean.

In [145]:
np.sum(pd.isna(products['review']))

829

In [146]:
products = products.fillna({'review': ''})

In [147]:
np.sum(pd.isna(products['review']))

0

In [148]:
def remove_punctuation(text):
    # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    table = str.maketrans({key: None for key in string.punctuation})
    return text.translate(table)

In [149]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [150]:
products.head()

,name,review,rating,review_clean
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3,These flannel wipes are OK but in my opinion n...
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...


In [151]:
products = products[products['rating'] != 3]

In [152]:
products['rating'].unique()

array([5, 4, 2, 1], dtype=int64)

In [153]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [154]:
products.head()['review_clean'].iloc[0]

'it came early and was not disappointed i love planet wise bags and now my wipe holder it keps my osocozy wipes moist and does not leak highly recommend it'

In [155]:
# IMPORTANT: If you are using a programming language with 1-based indexing (e.g. R, Matlab), 
# make sure to increment all indices by 1.
train_idx = pd.read_json('module-9-assignment-train-idx.json')
test_idx = pd.read_json('module-9-assignment-test-idx.json')
train_data = products.iloc[train_idx[0]]
test_data = products.iloc[test_idx[0]]

In [156]:
train_idx.head()

,0
0,0
1,1
2,2
3,3
4,4


### Build the word count vector for each review

We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

1. Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
2. Compute the occurrences of the words in each review and collect them into a row vector.
3. Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
4. Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.

In [157]:
from sklearn.feature_extraction.text import CountVectorizer

In [158]:
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

In [160]:
s = pd.Series(['aa f c', 'd c a'])
print(vectorizer.fit_transform(s).toarray())
print("-----------------------------------------------------------")
print(vectorizer.fit_transform(['A B C', 'D, b, F']).toarray())

[[0 1 1 0 1]
 [1 0 1 1 0]]
-----------------------------------------------------------
[[1 1 1 0 0]
 [0 1 0 1 1]]


### Train a sentiment classifier with logistic regression
Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model model.

In [161]:
from sklearn.linear_model import LogisticRegression

In [162]:
model = LogisticRegression()

In [163]:
model.fit(train_matrix, train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Model Evaluation
We will explore the advanced model evaluation concepts that were discussed in the lectures.

Accuracy

One performance metric we will use for our more advanced exploration is accuracy, which we have seen many times in past assignments. Recall that the accuracy is given by

$\mathrm{accuracy} = \frac{\mathrm{\#\, correctly\,calssified\, data\, point}}{ \mathrm{\#\, total\, data\, points}}$

Compute the accuracy on the test set using your tool of choice. If you are using scikit-learn, you can use the pre-defined method accuracy_score:

In [164]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true=test_data['sentiment'], y_pred=model.predict(test_matrix))
print( "Test Accuracy: %s" % accuracy)

Test Accuracy: 0.9321154307655387


In [165]:
# more convenient 
model.score(test_matrix, test_data['sentiment'])

0.9321154307655387

### Baseline: Majority class prediction

Recall from an earlier assignment that we used the majority class classifier as a baseline (i.e reference) model for a point of comparison with a more sophisticated classifier. The majority classifier model predicts the majority class for all data points.

Typically, a good model should beat the majority class classifier. Since the majority class in this dataset is the positive class (i.e., there are more positive than negative reviews), the accuracy of the majority class classifier is simply the fraction of positive reviews in the test set:

In [166]:
baseline = len(test_data[test_data['sentiment'] == 1])/len(test_data)
print( "Baseline accuracy (majority class classifier): %s" % baseline)

Baseline accuracy (majority class classifier): 0.8427825773938085


In [167]:
# Using accuracy as the evaluation metric, was our logistic regression model better than the baseline (majority class classifier)? ----> Yes

### Confusion Matrix

The accuracy, while convenient, does not tell the whole story. For a fuller picture, we turn to the confusion matrix. In the case of binary classification, the confusion matrix is a 2-by-2 matrix laying out correct and incorrect predictions made in each label as follows:

Using your tool, print out the confusion matrix for a classifier. For instance, scikit-learn provides the method confusion_matrix for this purpose:

In [168]:
from sklearn.metrics import confusion_matrix
cmat = confusion_matrix(y_true=test_data['sentiment'],
                        y_pred=model.predict(test_matrix),
                        labels=model.classes_)    # use the same order of class as the LR model.
print( ' target_label | predicted_label | count ')
print( '--------------+-----------------+-------')
# Print out the confusion matrix.
# NOTE: Your tool may arrange entries in a different order. Consult appropriate manuals.
for i, target_label in enumerate(model.classes_):
    for j, predicted_label in enumerate(model.classes_):
        print( '{0:^13} | {1:^15} | {2:5d}'.format(target_label, predicted_label, cmat[i,j]))

 target_label | predicted_label | count 
--------------+-----------------+-------
     -1       |       -1        |  3786
     -1       |        1        |  1455
      1       |       -1        |   808
      1       |        1        | 27287


#### Quiz Question: How many predicted values in the test set are false positives? ----> 1455

### Computing the cost of mistakes

Put yourself in the shoes of a manufacturer that sells a baby product on Amazon.com and you want to monitor your product's reviews in order to respond to complaints. Even a few negative reviews may generate a lot of bad publicity about the product. So you don't want to miss any reviews with negative sentiments --- you'd rather put up with false alarms about potentially negative reviews instead of missing negative reviews entirely. In other words, false positives cost more than false negatives. (It may be the other way around for other scenarios, but let's stick with the manufacturer's scenario for now.)

Suppose you know the costs involved in each kind of mistake:

1. 100 dollar for each false positive.
2. 1 dollar for each false negative.

Correctly classified reviews incur no cost.

Quiz Question: Given the stipulation, what is the cost associated with the logistic regression classifier's performance on the test set?

In [87]:
1455 * 100 + 808

146308

### Precision and Recall

You may not have exact dollar amounts for each kind of mistake. Instead, you may simply prefer to reduce the percentage of false positives to be less than, say, 3.5% of all positive predictions. This is where precision comes in:

$\mathrm{precision} = \frac{\mathrm{\#\, positive\, data\, points\,with\, positive\, predicitons}}{ \mathrm{\#\, all\, data\, points\, with\, positive\, predictions}} = \frac{\mathrm{\#\, true\, positives}}{{\#\, true positives} \, + \, {\#\, false\, positives}}$

So to keep the percentage of false positives below 3.5% of positive predictions, we must raise the precision to 96.5% or higher.

First, let us compute the precision of the logistic regression classifier on the test_data. Scikit-learn provides a predefined method for computing precision. (Consult appropriate manuals if you are using other tools.)


A complementary metric is recall, which measures the ratio between the number of true positives and that of (ground-truth) positive reviews:

$\mathrm{recall} = \frac{\mathrm{\#\, positive\, data\, points\,with\, positive\, predicitons}}{ \mathrm{\#\, all\, positive\, data\, points}} = \frac{\mathrm{\#\, true\, positives}}{{\#\, true positives} \, + \, {\#\, false\, negatives}}$

In [169]:
from sklearn.metrics import precision_score
precision = precision_score(y_true=test_data['sentiment'], y_pred=model.predict(test_matrix))
print( "Precision on test data: %s" % precision)

Precision on test data: 0.9493772180084893


#### Quiz Question: Out of all reviews in the test set that are predicted to be positive, what fraction of them are false positives? (Round to the second decimal place e.g. 0.25) 

In [170]:
round(1 - precision, 2)

0.05

#### Quiz Question: Based on what we learned in lecture, if we wanted to reduce this fraction of false positives to be below 3.5%, we would: (see quiz)

In [172]:
from sklearn.metrics import recall_score
recall = recall_score(y_true=test_data['sentiment'], y_pred=model.predict(test_matrix))
print( "Recall on test data: %s" % recall)

Recall on test data: 0.9712404342409682


Quiz Question: What fraction of the positive reviews in the test_set were correctly predicted as positive by the classifier?  0.97

Quiz Question: What is the recall value for a classifier that predicts +1 for all data points in the test_data?

### Precision-recall tradeoff

In this part, we will explore the trade-off between precision and recall discussed in the lecture. We first examine what happens when we use a different threshold value for making class predictions. We then explore a range of threshold values and plot the associated precision-recall curve.

Varying the threshold

False positives are costly in our example, so we may want to be more conservative about making positive predictions. To achieve this, instead of thresholding class probabilities at 0.5, we can choose a higher threshold.

Write a function called apply_threshold that accepts two things

1. probabilities: an SArray of probability values
2. threshold: a float between 0 and 1

The function should return an array, where each element is set to +1 or -1 depending whether the corresponding probability exceeds threshold.

Using the model you trained, compute the class probability values P(y=+1|x,w) for the data points in the test_data. Then use thresholds set at 0.5 (default) and 0.9 to make predictions from these probability values.

Note. If you are using scikit-learn, make sure to use predict_proba() function, not decision_function(). Also, note that the predict_proba() function returns the probability values for both classes +1 and -1. So make sure to extract the second column, which correspond to the class +1.

In [104]:
def apply_threshold(probabilities, threshold):
    return pd.Series(probabilities).apply(lambda x: 1 if x > threshold else -1).as_matrix()

In [107]:
prediction_1 = apply_threshold(model.predict_proba(test_matrix)[:, 1], 0.5)

f:\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [109]:
prediction_2 = apply_threshold(model.predict_proba(test_matrix)[:, 1], 0.9)

f:\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [113]:
np.sum(prediction_1 == 1)

28742

In [114]:
np.sum(prediction_2 == 1)

25076

### Exploring the associated precision and recall as the threshold varies

By changing the probability threshold, it is possible to influence precision and recall. Compute precision and recall for threshold values 0.5 and 0.9.

Quiz Question (variant 1): Does the precision increase with a higher threshold?

Quiz Question (variant 2): Does the recall increase with a higher threshold?

In [115]:
recall_score(test_data['sentiment'], y_pred=prediction_2)

0.8759921694251647

In [138]:
recall_score(test_data['sentiment'], y_pred=prediction_1)

0.9712404342409682

In [117]:
precision_score(test_data['sentiment'], y_pred=prediction_2)

0.9814563726272133

In [118]:
precision_score(test_data['sentiment'], y_pred=prediction_1)

0.9493772180084893

### Precision-recall curve

Now, we will explore various different values of tresholds, compute the precision and recall scores, and then plot the precision-recall curve. Use 100 equally spaced values between 0.5 and 1. In Python, we run

In [139]:
threshold_values = np.linspace(0.5, 1, num=100)
print(threshold_values)

[0.5        0.50505051 0.51010101 0.51515152 0.52020202 0.52525253
 0.53030303 0.53535354 0.54040404 0.54545455 0.55050505 0.55555556
 0.56060606 0.56565657 0.57070707 0.57575758 0.58080808 0.58585859
 0.59090909 0.5959596  0.6010101  0.60606061 0.61111111 0.61616162
 0.62121212 0.62626263 0.63131313 0.63636364 0.64141414 0.64646465
 0.65151515 0.65656566 0.66161616 0.66666667 0.67171717 0.67676768
 0.68181818 0.68686869 0.69191919 0.6969697  0.7020202  0.70707071
 0.71212121 0.71717172 0.72222222 0.72727273 0.73232323 0.73737374
 0.74242424 0.74747475 0.75252525 0.75757576 0.76262626 0.76767677
 0.77272727 0.77777778 0.78282828 0.78787879 0.79292929 0.7979798
 0.8030303  0.80808081 0.81313131 0.81818182 0.82323232 0.82828283
 0.83333333 0.83838384 0.84343434 0.84848485 0.85353535 0.85858586
 0.86363636 0.86868687 0.87373737 0.87878788 0.88383838 0.88888889
 0.89393939 0.8989899  0.9040404  0.90909091 0.91414141 0.91919192
 0.92424242 0.92929293 0.93434343 0.93939394 0.94444444 0.94949

#### For each of the values of threshold, we first obtain class predictions using that threshold and then compute the precision and recall scores. Save the precision scores and recall scores to lists precision_all and recall_all, respectively.

Let's plot the precision-recall curve to visualize the precision-recall tradeoff as we vary the threshold. Implement the function plot_pr_curve that generates a connected scatter plot from the lists of precision and recall scores. The function would be implemented in matplotlib as follows; for other tools, consult appropriate manuals.

In [121]:
def plot_pr_curve(precision, recall, title):
    plt.rcParams['figure.figsize'] = 7, 5
    plt.locator_params(axis = 'x', nbins = 5)
    plt.plot(precision, recall, 'b-', linewidth=4.0, color = '#B0017F')
    plt.title(title)
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.rcParams.update({'font.size': 16})

In [132]:
probabilities = model.predict_proba(test_matrix)[:, 1]

In [134]:
precision_all = []
recall_all = []

In [135]:
for i in threshold_values:
    pre = apply_threshold(probabilities, i)
    precision_all.append(precision_score(test_data['sentiment'], y_pred=pre))
    recall_all.append(recall_score(test_data['sentiment'], y_pred=pre))

f:\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
f:\anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Quiz Question: Among all the threshold values tried, what is the smallest threshold value that achieves a precision of 96.5% or better? Round your answer to 3 decimal places.

Quiz Question: Using threshold = 0.98, how many false negatives do we get on the test_data? This is the number of false negatives (i.e the number of reviews to look at when not needed) that we have to deal with using this classifier.

In [141]:
new_list = []
for i in zip(threshold_values, precision_all):
    thre, preci = i
    
    if preci >= 0.965:
        new_list.append(thre) 
        
print( min(new_list))

0.7070707070707071
